#### Goal
This file aimed to test the needed functions one by one to see weather they provide the proper output. 

##### Import needed packegaes

In [2]:
import matplotlib.pyplot as plt
from numba import jit 
import numpy as np 
import random 
import time 
from tqdm import tqdm 
from rich.progress import track 


##### Initializing perameters

In [ ]:
# Square 2D filed
# Periodic Boundary Conditions
'''
* * * ...
* * * ...
* * * ...
 ...  ...
'''
# J = 1 as default value
B = 0; # Magnetic field strength
L = 10; # Lattice size (width)
s_random = np.random.choice([1,-1],size=(L,L)) # Begin with random spin sites with values (+1 or -1) for up or down spins. 
s = s_random #Initial spin sites. Each element should be either +1 or -1, size should be L*L. Defualt value is randomized values.
n = 1000 * L**2 # number of MC sweeps 
Temperature = np.arange(1.6,3.25,0.01) # Initlaize temperature range (the range includes critical temperature) > takes form np.arange(start,stop,step)
  

##### calcE(s) 

In [ ]:
# test for calcE(S)
# input: surrent state s
# output: average energy for each element under s without outer magnetic field (lattice part)

def calcE_test():
    """
    testing for some special initial datas
    pass if all results have error less than 1e-10
    """
    
    test_cases = [
        {
            "name": "2×2 all up",
            "spins": np.array([[1, 1], [1, 1]]),
            "expected": -2.0
        },
        {
            "name": "2×2 checkerboard",
            "spins": np.array([[1, -1], [-1, 1]]),
            "expected": 2.0,
        },
        {
            "name": "2x2 single down",
            "spins": np.array([[1, 1], [1, -1]]),
            "expected": 0.0,
        },
        {
            "name": "2×2 strip",
            "spins": np.array([[1, 1, 1, 1],
                              [-1, -1, -1, -1]]),
            "expected": 0.0,
        },            
        {
            "name": "3×3 all up",
            "spins": np.ones((3, 3)),
            "expected": -2.0,
        },
        {
            "name": "3×3 checkerboard",
            "spins": np.array([[1, -1, 1], [-1, 1, -1], [1, -1, 1]]),
            "expected": 1/3,
        },
        {
            "name": "3×3 center down", 
            "spins": np.array([[1, 1, 1], [1, -1, 1], [1, 1, 1]]),
            "expected": -10/9,  
        },
        {
            "name": "1×1 up",
            "spins": np.array([[1]]),
            "expected": -2.0,
        },
        {
            "name": "4×4 chekerboard",
            "spins": np.array([[1, -1, 1, -1],
                              [-1, 1, -1, 1],
                              [1, -1, 1, -1],
                              [-1, 1, -1, 1]]),
            "expected": 2.0,
        },
        {
            "name": "4×4 strip",
            "spins": np.array([[1, 1, 1, 1],
                              [1, 1, 1, 1],
                              [-1, -1, -1, -1],
                              [-1, -1, -1, -1]]),
            "expected": -1.0,
        },
    ]
    
    
    all_passed = True    
    for i, case in enumerate(test_cases, 1):    
        global L
        L = case['spins'].shape[0]
        calculated = calcE(case["spins"])
        passed = abs(calculated - case["expected"]) < 1e-10
        
        print(f"test {i}: {case['name']}")
        print(f"  calculated value: {calculated:.1f}, expected value: {case['expected']:.1f}")
        print(f"  {'pass' if passed else 'fail'}\n")
        
        if not passed:
            all_passed = False
    
    print(f"\nresult: {'All tests have passed' if all_passed else 'failed'}")
    return all_passed

calcE_test()

test 1: 2×2 all up
  calculated value: -4.0, expected value: -2.0
  fail

test 2: 2×2 checkerboard
  calculated value: 4.0, expected value: 2.0
  fail

test 3: 2x2 single down
  calculated value: 0.0, expected value: 0.0
  pass

test 4: 2×2 strip
  calculated value: 0.0, expected value: 0.0
  pass

test 5: 3×3 all up
  calculated value: -4.0, expected value: -2.0
  fail

test 6: 3×3 checkerboard
  calculated value: 1.3, expected value: 0.3
  fail

test 7: 3×3 center down
  calculated value: -2.2, expected value: -1.1
  fail

test 8: 1×1 up
  calculated value: -4.0, expected value: -2.0
  fail

test 9: 4×4 chekerboard
  calculated value: 4.0, expected value: 2.0
  fail

test 10: 4×4 strip
  calculated value: -2.0, expected value: 0.0
  fail


result: failed


False

##### dE(s, i, j)

In [ ]:
# input: surrent state s, locatoin of the element i,j 
# output: interaction energy between spins which is difference in energy due to flipping spin i,j 

def dE_test():
    """
    testing for some special initial datas
    pass if all results have error less than 1e-10
    """
    
    test_cases = {
        "2x2_all_up": {
            "spins": np.ones((2, 2)),
            "name": "2x2 all up",
            "expected": {
                (0,0): 8.0, (0,1): 8.0,
                (1,0): 8.0, (1,1): 8.0
            }
        },
        
        "2x2_all_down": {
            "spins": -np.ones((2, 2)),
            "name": "2x2 all down", 
            "expected": {
                (0,0): 8.0, (0,1): 8.0,
                (1,0): 8.0, (1,1): 8.0
            }
        },
        
        "2x2_checkerboard": {
            "spins": np.array([[1, -1], 
                              [-1, 1]]),
            "name": "2x2_checkerboard",
            "expected": {
                (0,0): -8.0, (0,1): -8.0,
                (1,0): -8.0, (1,1): -8.0
            }
        },
        
        "2x2_single_down": {
            "spins": np.array([[1, 1], 
                              [1, -1]]),
            "name": "2x2 single down",
            "expected": {
                (0,0): 8.0, (0,1): 0.0,
                (1,0): 0.0, (1,1): -8.0
            }
        },
        
        "3x3_checkerboard": {
            "spins": np.array([[1, -1, 1],
                              [-1, 1, -1],
                              [1, -1, 1]]),
            "name": "3x3 checkerboard",
            "expected": {
                (0,0): 0.0, (0,1): -4.0, (0,2): 0.0,
                (1,0): -4.0, (1,1): -8.0, (1,2): -4.0,
                (2,0): 0.0, (2,1): -4.0, (2,2): 0.0
            }
        },
        
        "3x3_center_down": {
            "spins": np.array([[1, 1, 1],
                              [1, -1, 1],
                              [1, 1, 1]]),
            "name": "3x3 center down",
            "expected": {
                (0,0): 8.0, (0,1): 4.0, (0,2): 8.0,
                (1,0): 4.0, (1,1): -8.0, (1,2): 4.0,
                (2,0): 8.0, (2,1): 4.0, (2,2): 8.0
            }
        }
    }
    
    
    all_passed = True
    for name, data in test_cases.items():
        print(f"\n{data['name']}:")
        print(data['spins'])
        print()
        
        case_passed = True
        global L
        L = data['spins'].shape[0]
        
        for i in range(L):
            for j in range(L):
                calculated = dE(data['spins'], i, j)
                passed = abs(calculated - data['expected'][(i,j)]) < 1e-10
                
                print(f"test {(i,j)}")
                print(f"  calculated value: {calculated}, expected value: {data['expected'][(i,j)]}")
                print(f"  {'pass' if passed else '! fail'}\n")
                
                if not passed:
                    case_passed = False
        
        print(f"\ntest: {data['name']} {'pass' if case_passed else '! fail'}\n")
        print("=" * 50)
                
        if not case_passed: 
            all_passed = False   
            
    print("=" * 50)
    print(f"result: {'All tests have passed' if all_passed else '! failed'}")
    return all_passed

dE_test()


2x2 all up:
[[1. 1.]
 [1. 1.]]

test (0, 0)
  calculated value: 8.0, expected value: 8.0
  pass

test (0, 1)
  calculated value: 8.0, expected value: 8.0
  pass

test (1, 0)
  calculated value: 8.0, expected value: 8.0
  pass

test (1, 1)
  calculated value: 8.0, expected value: 8.0
  pass


test: 2x2 all up pass


2x2 all down:
[[-1. -1.]
 [-1. -1.]]

test (0, 0)
  calculated value: 8.0, expected value: 8.0
  pass

test (0, 1)
  calculated value: 8.0, expected value: 8.0
  pass

test (1, 0)
  calculated value: 8.0, expected value: 8.0
  pass

test (1, 1)
  calculated value: 8.0, expected value: 8.0
  pass


test: 2x2 all down pass


2x2_checkerboard:
[[ 1 -1]
 [-1  1]]

test (0, 0)
  calculated value: -8, expected value: -8.0
  pass

test (0, 1)
  calculated value: -8, expected value: -8.0
  pass

test (1, 0)
  calculated value: -8, expected value: -8.0
  pass

test (1, 1)
  calculated value: -8, expected value: -8.0
  pass


test: 2x2_checkerboard pass


2x2 single down:
[[ 1  1]
 [

True

##### mc(s,temp,n)

In [ ]:
def mc_test():
    '''
    since random function is hard to test, this function only test if mc() takes correct input and proper output.
    
    mc(s, temp, n):
    input: current state s, cuurent temp, number of MC sweeps
    outpur: next state s_mc
    '''
    test = True
    for i in range(30):
        global L
        L = random.randint(1,5)
        s = np.random.choice([1,-1],size=(L,L))
        n= 1000 * L**2
        temp = random.randrange(-200,200)
        s_mc = mc(s,temp,L)
        if not s_mc.size == s.size and type(s_mc) == 'numpy.ndarray':
            test = False
    
    print(f"{'This function has correct inout and output' if test else  '!! This function does not has correct inout and output'}")
    return test

mc_test()

This function has correct inout and output


True

##### physics(s,T,n)

In [104]:
def physics_test():
    ''' 
    since random function is hard to test, this function only test if mc() takes correct input and proper output.
    input: current state s, cuurent temp, number of MC sweeps
    output: average energy of lattice, average megnatic effect, CV
    '''
    test = True
    for i in range(30):
        global L
        L = random.randint(1,5)
        s = np.random.choice([1,-1],size=(L,L))
        n= 1000 * L**2
        temp = random.randrange(-200,200)
        En_avg, mag, CV = physics(s,temp,L)
        if not type(En_avg) == 'numpy.float64' and type(mag) == 'float' and type(CV) == 'numpy.float64':
            test = False
    
    print(f"{'This function has correct inout and output' if test else  '!! This function does not has correct inout and output'}")
    return test
physics_test()

This function has correct inout and output


True